In [1]:
import pandas as pd

df = pd.read_csv("../data/raw/lmsys/conversations_and_category_w_text.csv")

In [ ]:
df.columns


In [ ]:
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
import ast

def combine_conversation_steps(conversation):
    """Combines all steps of a conversation into a single string, keeping only content."""

    # remove outer brackets
    conversation = conversation.strip('[]')

    # convert to list of dict string, per line
    conversation = conversation.split('\n')

    # convert each line to dict
    conversation = [ast.literal_eval(line) for line in conversation]

    # Extract content from each step and join with newlines
    contents = [f"## Message by {step.get('role', '')}:\n\n{step.get('content', '')}\n\n\n\n" for step in conversation if isinstance(step, dict)]

    return '\n'.join(contents)

test_conversation = df["conversation_a"][1]
print(combine_conversation_steps(test_conversation))

In [5]:
# Apply the combine_conversation_steps function to both conversation columns
df['text_a'] = df['conversation_a'].apply(combine_conversation_steps)
df['text_b'] = df['conversation_b'].apply(combine_conversation_steps)


In [ ]:
df.columns

# Index(['index', 'question_id', 'model_a', 'model_b', 'winner',
#        'conversation_a', 'conversation_b', 'turn', 'anony', 'language',
#        'tstamp', 'conv_metadata', 'is_code', 'is_refusal', 'dedup_tag',
#        'category_tag', 'judge_hash', 'Prompt', 'Topic', 'broad_category_id',
#        'broad_category', 'narrower_category_id', 'narrower_category',
#        'prompt_count', 'prompt_percentage', 'example_prompt', 'response_a',
#        'response_b'],
#      dtype='object')


In [ ]:
print(df.winner.value_counts())

df_subset = df[df.winner.isin(['model_a', 'model_b'])]
# should be text_a if winner is model_a, otherwise text_b
df_subset["preferred_text"] = df_subset["winner"].map({"model_a": "text_a", "model_b": "text_b"})






In [ ]:
import datetime

# date string (nums only)
date_str = datetime.datetime.now().strftime("%Y-%m-%d")

save_path = f"../data/processed/lmsys/lmsys_arena_explorer_data_w_topics_{date_str}.csv"

# add winner_model and rejected_model columns
def get_winner_model(row):
    if row["winner"] == "model_a":
        return row["model_a"]
    else:
        return row["model_b"]

def get_rejected_model(row):
    if row["winner"] == "model_a":
        return row["model_b"]
    else:
        return row["model_a"]

df_subset["winner_model"] = df_subset.apply(get_winner_model, axis=1)
df_subset["rejected_model"] = df_subset.apply(get_rejected_model, axis=1)

# remove conversation columns
df_subset = df_subset.drop(columns=["conversation_a", "conversation_b"])
df_subset.to_csv(save_path, index=False)

In [ ]:
df_subset

In [ ]:
len(df_subset)